# <b><span style='color:#0B2F9F'>Pengantar</span></b>

Dalam pengelolaan data, sering kali informasi yang dibutuhkan tersebar di beberapa tabel yang saling berhubungan. Untuk memperoleh gambaran yang utuh, diperlukan cara untuk menggabungkan data-data tersebut secara tepat dan efisien. Konsep JOIN dalam SQL memungkinkan penggabungan data dari dua atau lebih tabel berdasarkan kolom yang memiliki relasi tertentu, sehingga informasi yang terpisah dapat ditampilkan dalam satu hasil query yang terpadu. Dengan memahami dan menerapkan berbagai jenis JOIN—seperti INNER JOIN, LEFT JOIN, RIGHT JOIN, dan FULL JOIN—seseorang dapat mengintegrasikan data dengan lebih fleksibel, memperkaya hasil analisis, serta meningkatkan ketepatan dalam pengambilan keputusan berbasis data.

# <b><span style='color:#0B2F9F'>Entity Relationalship Diagram</span></b>

Dalam sistem pemesanan pizza, terdapat beberapa entitas utama yang saling berhubungan. ERD (Entity Relationship Diagram) membantu menggambarkan hubungan antar tabel tersebut agar struktur data lebih mudah dipahami dan dikelola.

<img src="https://raw.githubusercontent.com/bachtiyarma/Material/refs/heads/main/Image/Materi-SQL/ERD%20-%20Pizza.png" width="60%">

Antar tabel bisa saling berelasi sesuai dengan peran dan hubungan logisnya melalui sebuah 'kunci'

# <b><span style='color:#0B2F9F'>Set up</span></b>

Dibutuhkan proses autentikasi dari Google Colab ke Google Big Query. Ikuti langkah berikut https://drive.google.com/file/d/1gW8alZ_PrvcrsieqWCHOR4ssLI_25BRc/view untuk detail step-by-step nya

In [ ]:
# Import library yang dibutuhkan
from google.colab import auth, data_table
from google.cloud import bigquery
from pandas_gbq import to_gbq
import pandas as pd
pd.set_option('display.float_format', '{:.2f}'.format)

# Proses autentikasi akun
auth.authenticate_user()
print('Authenticated')

In [ ]:
# Buat BigQuery client
project_id = 'dqlab-475713'
client = bigquery.Client(project = project_id)

# <b>A. <span style='color:#0B2F9F'><code>PREFIX</code></span></b>
Prefix merupakan penggunaan awalan pada nama tabel untuk memberikan informasi tambahan atau untuk mengelompokkan objek-objek tertentu. Sebagai contoh perhatikan gambar berikut : <br>

<img src="https://raw.githubusercontent.com/bachtiyarma/Material/refs/heads/main/Image/Materi-SQL/SQL%20-%20Prefix.png" width="50%">

<br>

Nama tabel dimisalkan (diberi alias) sebagai tbl, lalu untuk mengakses kolom pada tabel tersebut bisa diberi prefix tbl.kolom1 dan seterusnya

#### <i><b><span style='color:#55679C'>Quest</span> : Pada tabel customers, ambil kolom customer_id, name dan gender namun gunakan alias 'c' pada nama tabel dan sematkan prefixnya saat mengakses kolom!</b></i>

In [ ]:
%%bigquery --project {project_id} --verbose

SELECT
c.customer_id,
c.customer_name,
c.gender
FROM dqlab-9876543.dq_pizza.customers AS c

# <b>B. <span style='color:#0B2F9F'><code>JOIN</code></span></b>

Data biasanya disimpan di berbagai tabel atau sistem yang berbeda pada RDBMS (Relational Database Management System). JOIN memungkinkan pengguna untuk menggabungkan data dari berbagai sumber untuk mendapatkan gambaran yang lebih lengkap. Memahami query JOIN memungkinkan seorang pengguna untuk mengintegrasikan dan menganalisis data secara efektif, sehingga dapat meningkatkan kualitas analisis dan mendukung pengambilan keputusan berbasis data.

Operasi JOIN standar dapat dilakukan jika terdapat kunci pada tiap tabel yang saling terhubung. Operasi join dasar yang paling umum digunakan ada 4 yakni :
<ul>
    <li><b>LEFT JOIN</b></li>
    LEFT JOIN adalah jenis operasi penggabungan baris dari dua tabel, tetapi hanya mengembalikan semua baris dari tabel kiri (tabel pertama) dan baris yang cocok dari tabel kanan (tabel kedua). Jika tidak ada kecocokan di tabel kanan, maka nilai-nilai untuk kolom dari tabel kanan akan diisi dengan NULL.<br><br>
    <img src="https://thomasadventure.blog/img/left-join-extra.gif" width="20%">
    <br><br>
    <li><b>RIGHT JOIN</b></li>
    RIGHT JOIN adalah jenis operasi penggabungan baris dari dua tabel, tetapi hanya mengembalikan semua baris dari tabel kanan (tabel kedua) dan baris yang cocok dari tabel kiri (tabel pertama). Jika tidak ada kecocokan di tabel kiri, maka nilai-nilai untuk kolom dari tabel kiri akan diisi dengan NULL.<br><br>
    <img src="https://thomasadventure.blog/img/right-join.gif" width="20%">
    <br><br>
    <li><b>INNER JOIN</b></li>
    INNER JOIN adalah jenis operasi penggabungan yang hanya mengembalikan baris yang memiliki kecocokan di kedua tabel berdasarkan kriteria tertentu.<br><br>
    <img src="https://thomasadventure.blog/img/inner-join.gif" width="20%">
    <br><br>
     <li><b>FULL JOIN</b></li>
    FULL JOIN adalah jenis operasi penggabungan semua baris dari kedua tabel, baik yang memiliki kecocokan maupun yang tidak. Dengan demikian, hasilnya akan mencakup semua data dari tabel kiri dan tabel kanan.<br><br>
    <img src="https://thomasadventure.blog/img/full-join.gif" width="20%">
    <br><br>
</ul>

#### <i><b><span style='color:#55679C'>Quest : </span>Untuk mengetahui informasi mengenai banyak transaksi tiap customer (yang pernah bertransaksi saja); tampilkan customer_id, nama, gender, banyak transaksi yang pernah dilakukan urutkan berdasarkan transaksi terbanyak!

In [ ]:
%%bigquery --project {project_id} --verbose

SELECT distinct
  c.customer_id,
  c.customer_name,
  c.gender,
  count(t.order_id) as banyak_transaksi
FROM dqlab-9876543.dq_pizza.customers c
LEFT JOIN dqlab-9876543.dq_pizza.orders as t on t.customer_id = c.customer_id
WHERE t.order_id IS NOT NULL
group by c.customer_id, c.customer_name, c.gender
order by banyak_transaksi desc

In [ ]:
%%bigquery --project {project_id} --verbose

SELECT
c.customer_id, customer_name, gender, birth_date, phone_number, email

FROM (SELECT customer_id, customer_name, gender, birth_date, phone_number, email
FROM dqlab-9876543.dq_pizza.customers) AS c

LEFT JOIN (SELECT customer_id, count(*) as banyak_transaksi FROM dqlab-9876543.dq_pizza.orders GROUP BY customer_id)t
ON c.customer_id = t.customer_id
WHERE banyak_transaksi IS NULL

In [ ]:
%%bigquery --project {project_id} --verbose

SELECT distinct
  c.customer_id,
  c.customer_name,
  c.gender,
  count(distinct t.order_id) as banyak_transaksi
FROM dqlab-9876543.dq_pizza.customers c
INNER JOIN dqlab-9876543.dq_pizza.orders as t on t.customer_id = c.customer_id
group by c.customer_id, c.customer_name, c.gender
order by banyak_transaksi desc

#### <i><b><span style='color:#55679C'>Quest : </span>Kita perlu approch customer terdaftar namun belum pernah bertransaksi (belum pernah membeli pizza) di DQPizza, tampilkan identitas lengkap customer sesuai kriteria tersebut!

In [ ]:
%%bigquery --project {project_id} --verbose

SELECT
  a.customer_id,
  a.customer_name,
  a.gender,
  a.birth_date,
  a.phone_number,
  a.email
FROM dqlab-9876543.dq_pizza.customers AS a
FULL JOIN dqlab-9876543.dq_pizza.orders AS b
ON a.customer_id = b.customer_id
GROUP BY
  1, 2, 3, 4, 5, 6
HAVING COUNT(b.order_id) = 0


#### <i><b><span style='color:#55679C'>Quest : </span>Kategori pizza apa yang menyumbang pendapatan paling besar di DQPizza pada tahun 2024?

In [ ]:
%%bigquery --project {project_id} --verbose


SELECT
  p.pizza_id,
  p.pizza_type_id,
  p.size,
  p.price,
  CAST(REPLACE(p.price, 'IDR', '') AS FLOAT64) as price
FROM dqlab-9876543.dq_pizza.pizzas AS p;


In [ ]:
%%bigquery --project {project_id} --verbose

WITH temp_pizza AS (
  SELECT
    pizza_id,
    pizza_type_id,
    size,
    CAST(REPLACE(price, 'IDR', '') AS FLOAT64) AS price
  FROM dqlab-9876543.dq_pizza.pizzas
), temp_pizza_types AS (
  SELECT
    pizza_type_id,
    name,
    UPPER(category) AS category
  FROM dqlab-9876543.dq_pizza.pizza_types
)
SELECT
  pt.category,
  SUM(od.quantity * p.price) AS total_revenue
FROM dqlab-9876543.dq_pizza.order_details AS od
INNER JOIN temp_pizza AS p ON od.pizza_id = p.pizza_id
INNER JOIN temp_pizza_types AS pt ON p.pizza_type_id = pt.pizza_type_id
GROUP BY pt.category
ORDER BY total_revenue DESC

#### <i><b><span style='color:#55679C'>Quest : Tampilkan total pendapatan yang dihasilkan dari setiap pelanggan berdasarkan seluruh transaksi yang pernah dilakukan.

In [ ]:
%%bigquery --project {project_id} --verbose

SELECT
  c.customer_id,
  c.customer_name,
  c.gender,
  -- PERBAIKAN DI SINI:
  SUM(CAST(REPLACE(p.price, 'IDR', '') AS INT64) * od.quantity) AS total_pendapatan
FROM `dqlab-9876543.dq_pizza.customers` AS c
INNER JOIN `dqlab-9876543.dq_pizza.orders` AS o
  ON c.customer_id = o.customer_id
INNER JOIN `dqlab-9876543.dq_pizza.order_details` AS od
  ON o.order_id = od.order_id
INNER JOIN `dqlab-9876543.dq_pizza.pizzas` AS p
  ON od.pizza_id = p.pizza_id
GROUP BY c.customer_id, c.customer_name, c.gender
ORDER BY total_pendapatan DESC;


#### <i><b><span style='color:#55679C'>Quest : Pizza apa yang paling laku (best-seller)? Kita perlu tahu untuk manajemen stok dan promosi.

In [ ]:
%%bigquery --project {project_id} --verbose

SELECT
  pt.name AS nama_pizza,
  SUM(od.quantity) AS total_pesanan
FROM `dqlab-9876543.dq_pizza.order_details` AS od
INNER JOIN `dqlab-9876543.dq_pizza.pizzas` AS p
  ON od.pizza_id = p.pizza_id
INNER JOIN `dqlab-9876543.dq_pizza.pizza_types` AS pt
  ON p.pizza_type_id = pt.pizza_type_id
GROUP BY pt.name
ORDER BY total_pesanan DESC;


#### <i><b><span style='color:#55679C'>Quest : Seberapa sibuk toko kita? Secara rata-rata, ada berapa transaksi (pesanan) per harinya?

In [ ]:
%%bigquery --project {project_id} --verbose

SELECT
  o.order_date,
  COUNT(o.order_id) AS jumlah_transaksi,
  ROUND(AVG(COUNT(o.order_id)) OVER(), 2) AS rata_transaksi_harian
FROM `dqlab-9876543.dq_pizza.orders` AS o
GROUP BY o.order_date
ORDER BY o.order_date;

Data Source : <i>https://mavenanalytics.io/challenges/maven-pizza-challenge</i> (dengan modifikasi)


---

<br>
<a href="https://www.linkedin.com/in/muhamad-rasyid-aditya-17637a225/"><img src="https://img.shields.io/badge/-© 2025 Muhamad Rasyid Aditya-417DAC?style=for-the-badge&logoColor=white"/></a>

<a href="https://dqlab.id/"><img src="https://dqlab.id/files/dqlab/cache/87e30118ebba5ec7d96f6ea8c9dcc10b_x_118_X_55.png" align="left" /></a>
